In [1]:
import math

from uuid import uuid4

from vdom.svg import *

from sympy import symbols, sin, cos, pi, Point, Line, Point2D
from sympy.simplify import simplify



In [5]:
class ArchemedianSpiral:
    a, b, θ, ϕ = symbols('a,b,θ, ϕ')
    y1 = (a+b*θ)*sin(θ)
    x1 = (a+b*θ)*cos(θ)
    y2 = (a+b*ϕ)*sin(ϕ)
    x2 = (a+b*ϕ)*cos(ϕ)

    g = (b*sin(θ) + (a + b*θ)*cos(θ))/(b*cos(θ) - (a+b*θ)*sin(θ))
    h = (b*sin(ϕ) + (a + b*ϕ)*cos(ϕ))/(b*cos(ϕ) - (a+b*ϕ)*sin(ϕ))

    p1 = Point(x1,y1)
    p2 = Point(x2,y2)
    l1 = Line(p1, slope=g)
    l2 = Line(p2, slope=h)
    #self.int_point = self.l1.intersection(self.l2)[0]
    # hard code this so that it takes less time to create the class
    int_point = Point2D(
        (a**3*sin(θ) - a**3*sin(ϕ) + a**2*b*θ*sin(θ) - 2*a**2*b*θ*sin(ϕ) + 2*a**2*b*ϕ*sin(θ) 
         - a**2*b*ϕ*sin(ϕ) - a**2*b*cos(θ) + a**2*b*cos(ϕ) - a*b**2*θ**2*sin(ϕ) + 2*a*b**2*θ*ϕ*sin(θ) 
         - 2*a*b**2*θ*ϕ*sin(ϕ) + 2*a*b**2*θ*cos(ϕ) + a*b**2*ϕ**2*sin(θ) - 2*a*b**2*ϕ*cos(θ) - b**3*θ**2*ϕ*sin(ϕ) 
         + b**3*θ**2*cos(ϕ) + b**3*θ*ϕ**2*sin(θ) - b**3*ϕ**2*cos(θ))
        /(a**2*sin(θ - ϕ) + a*b*θ*sin(θ - ϕ) + a*b*ϕ*sin(θ - ϕ) + b**2*θ*ϕ*sin(θ - ϕ) + b**2*θ*cos(θ - ϕ) - 
          b**2*ϕ*cos(θ - ϕ) + b**2*sin(θ - ϕ)), 
        (((a + b*θ)
          *(b*cos(θ) - (a + b*θ)*sin(θ))
          *(a**2*sin(θ - ϕ) + a*b*θ*sin(θ - ϕ) + a*b*ϕ*sin(θ - ϕ) + b**2*θ*ϕ*sin(θ - ϕ) + b**2*θ*cos(θ - ϕ) - b**2*ϕ*cos(θ - ϕ) + b**2*sin(θ - ϕ))
          *sin(θ)) 
         - ((b*sin(θ) + (a + b*θ)*cos(θ))
            *(-(b*sin(θ) + (a + b*θ)*cos(θ))
              *(b*cos(ϕ) - (a + b*ϕ)*sin(ϕ))
              *((a + b*θ)*cos(θ) - (a + b*ϕ)*cos(ϕ)) 
              + ((b*cos(θ) - (a + b*θ)*sin(θ))
                *(b*cos(ϕ) - (a + b*ϕ)*sin(ϕ))
                *((a + b*θ)*sin(θ) - (a + b*ϕ)*sin(ϕ)))
              + ((a + b*θ)*cos(θ) - (a + b*ϕ)*cos(ϕ))
              *(a**2*sin(θ - ϕ) 
                + a*b*θ*sin(θ - ϕ) 
                + a*b*ϕ*sin(θ - ϕ) 
                + b**2*θ*ϕ*sin(θ - ϕ) 
                + b**2*θ*cos(θ - ϕ) 
                - b**2*ϕ*cos(θ - ϕ) 
                + b**2*sin(θ - ϕ))
             ))
        )/((b*cos(θ) - (a + b*θ)*sin(θ))
           *(a**2*sin(θ - ϕ) 
             + a*b*θ*sin(θ - ϕ) 
             + a*b*ϕ*sin(θ - ϕ) 
             + b**2*θ*ϕ*sin(θ - ϕ) 
             + b**2*θ*cos(θ - ϕ) 
             - b**2*ϕ*cos(θ - ϕ) 
             + b**2*sin(θ - ϕ))
          )
    )
    
    def __init__(self, a=0, b=1):
        self.a = a
        self.b = b
    
    @property
    def initial_point(self):
        """ Because quadratic curves are assuming that they are absolute we need to provide the starting point.
        """
        return list(map(float, self.p1.subs({"a":self.a, 
                                             "b":self.b, 
                                             "θ":0
                                            }).args))

    
    def get_intersection_points(self, num_cycles, angle_change, θ0=0):
        """
        num_cycles: 
            how many times does the spiral go around (this will be rounded up to give a complete segment)
        angle_change:
            how large is the change in angle for each quadratic segment
        """
        num_degs = num_cycles*360
        num_angles = math.floor(num_degs/angle_change)
        

        δθ = (pi/180)*angle_change
        for x in range(num_angles):
            loc_θ = θ0 + x*δθ
            yield {"int_point": self.int_point.subs({"a":self.a, 
                                                     "b":self.b, 
                                                     "θ":loc_θ, 
                                                     "ϕ":(loc_θ+δθ)}),
                   "loc_θ": loc_θ,
                   "δθ": δθ
                  }

    def get_quad_control_points(self, num_cycles, angle, θ0=0):
        
        for x in self.get_intersection_points(num_cycles, angle, θ0=θ0):
            yield (tuple(map(float, x['int_point'].args)), 
                   tuple(map(float, self.p2.subs({"a":self.a, 
                                                  "b":self.b, 
                                                  "θ":x['loc_θ'], 
                                                  "ϕ":(x['loc_θ']+x['δθ'])
                                                 }).args))
                  )
            
#     def _html_repr_(self):
#         center = 500
#         d = f'M{center},{center} {" ".join("Q"+",".join(map(str, (q+center for q in quad[0])))+" "+",".join(map(str, (q+center for q in quad[1]))) for quad in quads)}'
#         myid = "blah"

#         svg(path(id="blah",
#                  fill="none", 
#                  stroke="red",
#                  d=d
#                 ),
#             text(
#                 textPath("The quick brown fox jumps over the lazy dog. "*10,
#                          **{"xlink:href":"#blah"}),

#             ),
#             width="1000",
#             height="1000"
#         )

        
            

In [3]:

# quads = [((4.886069913362311, 0.0), (5.553603672697958, 5.553603672697958)),
#  ((6.292624949580308, 11.701953804952588), (0.0, 15.707963267948966)),
#  ((-7.384659055703452, 20.409183235003095),
#   (-16.660811018093874, 16.660811018093874)),
#  ((-27.507821040299458, 12.277675514426061), (-31.41592653589793, 0.0)),
#  ((-35.89841593302279, -14.082155759801585),
#   (-27.768018363489787, -27.768018363489787)),
#  ((-18.616080799648483, -43.173434763497326), (0.0, -47.1238898038469)),
#  ((20.633219995134468, -51.50239507601781),
#   (38.87522570888571, -38.87522570888571)),
#  ((58.84465063940477, -25.05233173995045), (62.83185307179586, 0.0)),
#  ((67.15332323416332, 27.15259782960218),
#   (49.98243305428162, 49.98243305428162)),
#  ((31.522407781964347, 74.52625872641839), (0.0, 78.53981633974483)),
#  ((-33.662801829667465, 82.8258973873564),
#   (-61.089640399677535, 61.089640399677535)),
#  ((-90.21509001782945, 38.00720396917751), (-94.2477796076938, 0.0)),
#  ((-98.5099327531225, -40.169847030460346),
#   (-72.19684774507346, -72.19684774507346)),
#  ((-44.49949989472333, -105.90870551943563), (0.0, -109.95574287564277)),
#  ((46.67571405557359, -114.200697512379),
#   (83.30405509046936, -83.30405509046936)),
#  ((121.60556841042049, -50.99606275914522), (125.66370614359172, 0.0)),
#  ((129.89572745490608, 53.181148245842266),
#   (94.41126243586528, 94.41126243586528)),
#  ((57.495257304576675, 137.30471189849013), (0.0, 141.3716694115407)),
#  ((-59.68645381284364, 145.5936223715141),
#   (-105.5184697812612, 105.5184697812612)),
#  ((-153.0055094831808, 63.99617751671841), (-157.07963267948966, 0.0)),
#  ((-161.293530976059, -66.19175965738393),
#   (-116.62567712665711, -116.62567712665711)),
#  ((-70.49828488840743, -168.70754121307837), (0.0, -172.78759594743863)),
#  ((72.69711967853593, -176.99490720030485),
#   (127.73288447205303, -127.73288447205303)),
#  ((184.4105165053009, -77.0012410056532), (188.4955592153876, 0.0)),
#  ((192.69738521427902, 79.20255413867925),
#   (138.84009181744895, 138.84009181744895)),
#  ((83.5048233608484, 200.1142285110613), (0.0, 204.20352248333657)),
#  ((-85.70806788295005, 208.4007109960834),
#   (-149.94729916284487, 149.94729916284487)),
#  ((-215.8185262693156, 90.0088800799815), (-219.91148575128554, 0.0)),
#  ((-224.10470278315773, -92.21365875566124),
#   (-161.05450650824076, -161.05450650824076)),
#  ((-96.51330419152706, -231.52329716957763), (0.0, -235.61944901923448)),
#  ((98.71932161241247, -239.8092271561129),
#   (172.16171385363668, -172.16171385363668)),
#  ((247.22845557688547, -103.01801830763809), (251.32741228718345, 0.0)),
#  ((255.514184008721, 105.22505026111878),
#   (183.2689211990326, 183.2689211990326)),
#  ((109.52296513732743, 262.9339352530884), (0.0, 267.0353755551324)),
#  ((-111.73083839545508, 271.21949678488266),
#   (-194.37612854442852, 194.37612854442852)),
#  ((-278.6396841838072, 116.0281014085204), (-282.7433388230814, 0.0)),
#  ((-286.9251059521982, -118.23668002670851),
#   (-205.48333588982445, -205.48333588982445)),
#  ((-122.53339385744391, -294.345660969717), (0.0, -298.45130209103036)),
#  ((124.74256966311573, -302.63096452805564),
#   (216.59054323522037, -216.59054323522037)),

#  ((310.0518322592328, -129.0388165127512), (314.1592653589793, 0.0))]

In [35]:
a = 45
b = 5 # number of units between cycles
num_cycles = 3 # minimum number of cycles to be covered
angle = 45 # number of degrees each quadratic curve will approximate

blah = ArchemedianSpiral(a=a, b=b)
quad_iter = blah.get_quad_control_points(num_cycles=num_cycles, angle=angle)
quads = list(quad_iter)

font_size = 12 # in px, 14 matches notebook default

center = float(b*(num_cycles*pi*2)+font_size) + max(blah.initial_point)

starting_point = map(lambda x: str(center + x), blah.initial_point)
d = f'M{",".join(starting_point)} {" ".join("Q"+",".join(map(str, (q+center for q in quad[0])))+" "+",".join(map(str, (q+center for q in quad[1]))) for quad in quads)}'
myid = f"{uuid4()}".replace("-","")

svg(path(id=f"{myid}",
         fill="none", 
         stroke="red",
         d=d
        ),
    text(
        textPath("The quick brown fox jumps over the lazy dog. "*10,
                 **{"xlink:href":f"#{myid}"}),
        style={"fontSize": f"{font_size}px"},
    ),
    width=f"{center*2 + 1.5*font_size}",
    height=f"{center*2 + 1.5*font_size}"
)


<svg height="320.49555921538763" width="320.49555921538763"><path d="M196.24777960769381,151.24777960769381 Q198.38168873620214,170.45296176426857 185.84438659743745,185.84438659743745 Q172.6194414971208,202.07999663390132 151.24777960769381,204.1017612416683 Q128.78955632676238,206.22631483125969 111.09756894525225,191.39799027013538 Q92.56751319722423,175.86724887617265 90.53981633974485,151.24777960769381 Q88.42222115104161,125.53680142125738 105.54396527255429,105.54396527255429 Q123.37901345527828,84.71821332282107 151.24777960769381,82.68583470577036 Q180.21141923337532,80.5736105217049 202.5051976155313,99.99036159985633 Q225.62754519085217,120.1287544517317 227.66370614359175,151.24777960769381 Q229.77166257775377,183.46406841045348 208.05880128822926,208.05880128822926 Q185.6177178174334,233.478410711994 151.24777960769381,235.51768777756624 Q115.77881959771447,237.62217328017772 88.88315425446041,213.61240496092722 Q61.165776792405055,188.86909399286148 59.12388980384692,151.24777960769381 Q57.02228148060641,112.52611289148263 83.32955058176245,83.32955058176245 Q110.37475004275191,53.3140119789019 151.24777960769381,51.26990816987244 Q193.22219248437244,49.17072300654439 224.71961230632314,77.77594690906449 Q257.0336231483308,107.12277977820979 259.0796326794897,151.24777960769381 Q261.1767494780372,196.47497770676767 230.27321597902107,230.27321597902107 Q198.62494614861617,264.8859493309095 151.24777960769381,266.93361431346415 Q102.76775951844621,269.0289451814373 66.66873956366858,235.82681965171906 Q29.757079458464517,201.87726717471116 27.707963267948983,151.24777960769381 Q25.614189885222714,99.51490377588404 61.11513589097062,61.11513589097062 Q97.36584770702706,21.90438023465441 151.24777960769381,19.853981633974513 Q206.23354176792412,17.76157837719262 246.93402699711498,55.56153221827266 Q288.44401927688364,94.11330336505708 290.49555921538763,151.24777960769381" fill="none" id="d136a542bec34056ba0e297bbed01ab7" stroke="red"></path><text style="font-size: 12px"><textPath xlink:href="#d136a542bec34056ba0e297bbed01ab7">The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. </textPath></text></svg>

In [24]:
display(max(max(map(abs,quad[1])) for quad in quads))
display(quads)

139.2477796076938

[((47.13390912850831, 19.205182156574757),
  (34.59660698974362, 34.59660698974362)),
 ((21.37166188942698, 50.832217026207516), (0.0, 52.853981633974485)),
 ((-22.458223280931445, 54.97853522356586),
  (-40.15021066244157, 40.15021066244157)),
 ((-58.68026641046958, 24.619469268478834), (-60.70796326794897, 0.0)),
 ((-62.82555845665221, -25.71097818643644),
  (-45.70381433513953, -45.70381433513953)),
 ((-27.86876615241553, -66.52956628487274), (0.0, -68.56194490192345)),
 ((28.9636396256815, -70.67416908598892),
  (51.257418007837494, -51.257418007837494)),
 ((74.37976558315836, -31.11902515596212), (76.41592653589794, 0.0)),
 ((78.52388297005996, 32.21628880275966),
  (56.81102168053545, 56.81102168053545)),
 ((34.369938209739594, 82.23063110430019), (0.0, 84.26990816987242)),
 ((-35.46896000997934, 86.37439367248389),
  (-62.36462535323341, 62.36462535323341)),
 ((-90.08200281528876, 37.62131438516766), (-92.1238898038469, 0.0)),
 ((-94.2254981270874, -38.72166671621118),
  (-67.91

In [41]:
def svgStyledSpiral(content = None,a=0, b=5, num_cycles=3, angle=45, font_size=1):
    if content is None:
        content = "The quick brown fox jumps over the lazy dog. "*10
#     a = a
#     b = b # number of units between cycles
#     num_cycles = num_cycles # minimum number of cycles to be covered
#     angle = angle # number of degrees each quadratic curve will approximate

    blah = ArchemedianSpiral(a=a, b=b)
    quad_iter = blah.get_quad_control_points(num_cycles=num_cycles, angle=angle)
    quads = list(quad_iter)

    font_size = font_size # 1 matches notebook default, in em
    center = float(b*(num_cycles*pi*2)+font_size*14) + max(blah.initial_point)
    starting_point = map(lambda x: str(center + x), blah.initial_point)

    d = f'M{",".join(starting_point)} {" ".join("Q"+",".join(map(str, (q+center for q in quad[0])))+" "+",".join(map(str, (q+center for q in quad[1]))) for quad in quads)}'
    myid = f"{uuid4()}".replace("-","")

    return svg(path(id=f"{myid}",
             fill="none", 
             stroke="red",
             d=d
            ),
        text(
            textPath(content,
                     **{"xlink:href":f"#{myid}"}),
            style={"fontSize": f"{font_size}em",
                   "fontFamily": "Monospace",
                  }
        ),
        width=f"{center*2 + 1.5*font_size}",
        height=f"{center*2 + 1.5*font_size}"
    )


In [34]:
def svgStyledSpiral(content = None, a=0, b=5, num_cycles=3, angle=45, font_size=1, stroke="", quads=None):
    if content is None:
        content = "The quick brown fox jumps over the lazy dog. "*10

    blah = ArchemedianSpiral(a=a, b=b, num_cycles=num_cycles, angle=angle)
    quad_iter = blah.get_quad_control_points(num_cycles=num_cycles, angle=angle)
    quads = list(quad_iter)

#   a
#   b # number of units between cycles
#   num_cycles # minimum number of cycles to be covered
#   angle # number of degrees each quadratic curve will approximate

#   font_size # 1 matches notebook default, in em

    center = float(b*(num_cycles*pi*2)+font_size*14) + max(blah.initial_point)
    center = blah.center
    starting_point = map(lambda x: str(center + x), blah.initial_point)

    d = f'M{",".join(starting_point)} {" ".join("Q"+",".join(map(str, (q+center for q in quad[0])))+" "+",".join(map(str, (q+center for q in quad[1]))) for quad in quads)}'
    myid = f"{uuid4()}"

    return svg(path(id=f"{myid}",
             fill="none", 
             stroke=stroke,
             d=d
            ),
        text(
            textPath(content,
                     **{"xlink:href":f"#{myid}"}),
            style={"fontSize": f"{font_size}em",
                   "fontFamily": "Monospace",
                  }
        ),
        width=f"{center*2 + 1.5*font_size}",
        height=f"{center*2 + 1.5*font_size}"
    )


In [103]:
len("The quick brown fox jumps over the lazy dog. 12345")

50

In [88]:
def numstr(m):
    return "".join(str(x)[-1] for x in range(m)).replace("0","|")
    
# 45+35
45*2+36

126

In [100]:
for x in range(5):
    display(svgStyledSpiral(content= "The quick brown fox jumps over the lazy dog. 12345"*4,a=60, b=1.75, num_cycles=x, angle=45, font_size=1))

<svg height="149.5" width="149.5"><path d="M134.0,74.0 " fill="none" id="4a935d51fac842528f13ea1391671f4d" stroke="red"></path><text style="font-family: Monospace; font-size: 1em"><textPath xlink:href="#4a935d51fac842528f13ea1391671f4d">The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345</textPath></text></svg>

<svg height="171.49114857512856" width="171.49114857512856"><path d="M144.9955742875643,84.99557428756428 Q145.72615766129667,110.04414710124657 128.39386180147926,128.39386180147926 Q110.79581565703042,147.02492652850205 84.99557428756428,147.74446785945534 Q58.808445830392685,148.4747990603959 39.653525488205005,130.33762308692354 Q20.21754905248251,111.93432416771228 19.497787143782148,84.99557428756428 Q18.767686814081458,57.669885151369414 37.70976420276072,37.70976420276072 Q56.91830517361507,17.468858564095115 84.99557428756428,16.748893571891074 Q113.45982865792168,16.019005398198757 134.22514565781213,35.766002917316435 Q155.27099599509324,55.77977657654853 155.99114857512856,84.99557428756428" fill="none" id="880a525d6ff14495849bed773b2045b7" stroke="red"></path><text style="font-family: Monospace; font-size: 1em"><textPath xlink:href="#880a525d6ff14495849bed773b2045b7">The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345</textPath></text></svg>

<svg height="193.48229715025712" width="193.48229715025712"><path d="M155.99114857512856,95.99114857512856 Q156.72173194886096,121.03972138881085 139.38943608904356,139.38943608904356 Q121.7913899445947,158.02050081606632 95.99114857512856,158.74004214701964 Q69.80402011795697,159.4703733479602 50.64909977576929,141.33319737448784 Q31.213123340046792,122.92989845527657 30.49336143134643,95.99114857512856 Q29.76326110164574,68.6654594389337 48.705338490325,48.705338490325 Q67.91387946117935,28.464432851659396 95.99114857512856,27.744467859455355 Q124.45540294548596,27.01457968576304 145.2207199453764,46.761577204880716 Q166.26657028265754,66.77535086411281 166.98672286269283,95.99114857512856 Q167.71641541122048,125.59397231002932 147.16448123082068,147.16448123082068 Q126.34548310587297,169.01529005661018 95.99114857512856,169.7356164345839 Q65.24975172148268,170.46512803125592 42.874054633992145,149.10824251626497 Q20.218274994432065,127.48402718464722 19.497787143782148,95.99114857512856 Q18.76844341520986,64.11117518234803 40.930293348547856,40.930293348547856 Q63.35971944730912,17.46953183453708 95.99114857512856,16.748893571891074 Q129.0097016305707,16.019705995835054 152.99576508715356,38.986532063103574 Q177.26151843877813,62.22116319074903 177.98229715025712,95.99114857512856" fill="none" id="b2f2b11c8c4748dc930a98c45d958b87" stroke="red"></path><text style="font-family: Monospace; font-size: 1em"><textPath xlink:href="#b2f2b11c8c4748dc930a98c45d958b87">The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345</textPath></text></svg>

<svg height="215.47344572538566" width="215.47344572538566"><path d="M166.98672286269283,106.98672286269283 Q167.71730623642523,132.0352956763751 150.38501037660782,150.38501037660782 Q132.78696423215897,169.01607510363058 106.98672286269283,169.7356164345839 Q80.79959440552123,170.46594763552446 61.644674063333554,152.3287716620521 Q42.20869762761106,133.92547274284084 41.488935718910696,106.98672286269283 Q40.758835389210006,79.66103372649796 59.700912777889265,59.700912777889265 Q78.90945374874362,39.46000713922366 106.98672286269283,38.74004214701962 Q135.45097723305022,38.010153973327306 156.21629423294067,57.75715149244498 Q177.2621445702218,77.77092515167708 177.98229715025712,106.98672286269283 Q178.71198969878475,136.5895465975936 158.16005551838498,158.16005551838498 Q137.34105739343724,180.01086434417442 106.98672286269283,180.73119072214817 Q76.24532600904695,181.4607023188202 53.86962892155641,160.10381680382926 Q31.213849281996332,138.4796014722115 30.493361431346415,106.98672286269283 Q29.764017702774126,75.1067494699123 51.92586763611212,51.92586763611212 Q74.35529373487338,28.465106122101346 106.98672286269283,27.74446785945534 Q140.00527591813497,27.01528028339932 163.99133937471782,49.98210635066784 Q188.25709272634236,73.21673747831329 188.97787143782136,106.98672286269283 Q189.70691339294189,141.14385843971348 165.9351006601621,165.9351006601621 Q141.8952696383804,191.00585485543775 106.98672286269283,191.72676500971244 Q71.69100214159525,192.4556708456897 46.09458377977927,167.8788619456064 Q20.218820573277384,143.03383564229017 19.497787143782134,106.98672286269283 Q18.7690088250271,70.55241458675283 44.15082249433498,44.15082249433498 Q69.80103992057634,17.470042846648028 106.98672286269283,16.748893571891074 Q144.5596209141666,16.020234958238206 171.76638451649495,42.20706120889069 Q199.25218738385098,68.66246599601284 199.97344572538566,106.98672286269283" fill="none" id="83f08ee2a6484acca8fda54b7d73f23a" stroke="red"></path><text style="font-family: Monospace; font-size: 1em"><textPath xlink:href="#83f08ee2a6484acca8fda54b7d73f23a">The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345</textPath></text></svg>

<svg height="237.4645943005142" width="237.4645943005142"><path d="M177.9822971502571,117.9822971502571 Q178.7128805239895,143.03086996393938 161.3805846641721,161.3805846641721 Q143.78253851972323,180.01164939119485 117.9822971502571,180.73119072214817 Q91.7951686930855,181.46152192308872 72.64024835089782,163.32434594961637 Q53.204271915175326,144.9210470304051 52.48451000647496,117.9822971502571 Q51.75440967677427,90.65660801406223 70.69648706545354,70.69648706545354 Q89.90502803630788,50.45558142678793 117.9822971502571,49.73561643458389 Q146.4465515206145,49.00572826089157 167.21186852050494,68.75272578000926 Q188.25771885778607,88.76649943924134 188.97787143782136,117.9822971502571 Q189.70756398634902,147.58512088515786 169.15562980594922,169.15562980594922 Q148.3366316810015,191.00643863173872 117.9822971502571,191.72676500971244 Q87.24090029661122,192.45627660638445 64.86520320912068,171.0993910913935 Q42.2094235695606,149.47517575977577 41.48893571891068,117.9822971502571 Q40.75959199033839,86.10232375747657 62.92144192367639,62.92144192367639 Q85.35086802243765,39.46068040966561 117.9822971502571,38.74004214701961 Q151.00085020569924,38.01085457096359 174.9869136622821,60.97768063823211 Q199.25266701390666,84.21231176587756 199.97344572538566,117.9822971502571 Q200.70248768050612,152.13943272727775 176.93067494772637,176.93067494772637 Q152.8908439259447,202.001429143002 117.9822971502571,202.72233929727673 Q82.68657642915952,203.45124513325396 57.09015806734354,178.87443623317066 Q31.21439486084165,154.02940992985447 30.4933614313464,117.9822971502571 Q29.764583112591367,81.5479888743171 55.14639678189925,55.14639678189925 Q80.79661420814061,28.465617134212295 117.9822971502571,27.74446785945534 Q155.55519520173087,27.015809245802473 182.76195880405925,53.20263549645496 Q210.24776167141525,79.65804028357711 210.96902001294993,117.9822971502571 Q211.69756603757116,156.6937870269263 184.70572008950353,184.70572008950353 Q157.44513135563835,212.9965523774725 117.9822971502571,213.717913584841 Q78.13221355297719,214.4463535207022 49.315112925566396,186.6494813749478 Q20.219245529835973,158.58371180201453 19.497787143782134,117.9822971502571 Q18.769447343192354,76.99361807637274 47.371351640122114,47.371351640122114 Q76.24229921543962,17.47044390808405 117.9822971502571,16.74889357189106 Q160.1095733304433,16.020648440250724 190.53700394583637,45.42759035467782 Q221.2429568321075,75.10371333618598 221.9645943005142,117.9822971502571" fill="none" id="a420149589324ef78c6cf6777c9d5ef1" stroke="red"></path><text style="font-family: Monospace; font-size: 1em"><textPath xlink:href="#a420149589324ef78c6cf6777c9d5ef1">The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345The quick brown fox jumps over the lazy dog. 12345</textPath></text></svg>

In [ ]:
# 19 chars for a = 20 num_cycles=1 b=1.75 font_size=1
# 19+27 for a =20 num_cycles=2 b=1.75 font_size=1

In [102]:
(float((60+1.75)*pi))/49

3.9590478848810147

In [94]:
def circ(r):
    return float(2*pi*r)



display(f"1 turn: + {circ(20+1.75)/19}")
display(f"2 turn: + {sum(circ(20+1.75*x) for x in range(2))/(46)}")
display(f"3 turn: + {sum(circ(20+1.75*x) for x in range(3))/(80)}")
display(f"4 turn: + {sum(circ(20+1.75*x) for x in range(4))/(126)}")




'1 turn: + 7.192593706902947'

'2 turn: + 5.702673621190167'

'3 turn: + 5.12472301616835'

'4 turn: + 4.512922780156766'

126

In [ ]:
19

In [13]:
text("hi", style={"fontSize":"12px"})

<text style="font-size: 12px">hi</text>

In [45]:
quads

[((2.4430349566811556, 0.0), (2.776801836348979, 2.776801836348979)),
 ((3.146312474790154, 5.850976902476294), (0.0, 7.853981633974483)),
 ((-3.692329527851726, 10.204591617501547),
  (-8.330405509046937, 8.330405509046937)),
 ((-13.753910520149729, 6.138837757213031), (-15.707963267948966, 0.0)),
 ((-17.949207966511395, -7.041077879900793),
  (-13.884009181744894, -13.884009181744894)),
 ((-9.308040399824241, -21.586717381748663), (0.0, -23.56194490192345)),
 ((10.316609997567234, -25.751197538008906),
  (19.437612854442854, -19.437612854442854)),
 ((29.422325319702384, -12.526165869975225), (31.41592653589793, 0.0)),
 ((33.57666161708166, 13.57629891480109),
  (24.99121652714081, 24.99121652714081)),
 ((15.761203890982173, 37.263129363209195), (0.0, 39.269908169872416)),
 ((-16.831400914833733, 41.4129486936782),
  (-30.544820199838767, 30.544820199838767)),
 ((-45.10754500891473, 19.003601984588755), (-47.1238898038469, 0.0)),
 ((-49.25496637656125, -20.084923515230173),
  (-36.098

In [50]:
" ".join("q"+",".join(" ".join(",".join(map(str, ps) for ps in quad)) for quad in quads))

TypeError: sequence item 0: expected str instance, map found

In [ ]:
center = 100
" ".join("q"+",".join(map(str, (q+center for q in quad[0])))+" "+",".join(map(str, (q+center for q in quad[1]))) for quad in quads)

# [" ".join("q"+",".join(map(str, ps)) for ps in quad) for quad in quads]

In [ ]:
from vdom.logos import nteract


In [ ]:

jupyter()